In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

As I found that I had an issue with get_shared_words - function can return the same word from text1 multiple times even though in text2 it appears only once (cause I didn't deleted it from both texts once match found)

This notebook is execution of clean algo with fixed function, in order to be able to compare if there is main difference in results

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder

import sys
import imp

sys.path.append('../src/')
import data.reader as dataReader
import utils.utils as thesisUtils
import similarities.cosine as thesisCosineSimilarity
import vocabulary.vocabulary as thesisVocabulary
import features.model_features as thesisModelFeatures
import features.factory as thesisFactoryFeatures
import data.corpus_stats as thesisCorpusStats

imp.reload(dataReader)
imp.reload(thesisUtils)
imp.reload(thesisVocabulary)
imp.reload(thesisCorpusStats)
imp.reload(thesisModelFeatures)
imp.reload(thesisFactoryFeatures)
imp.reload(thesisCosineSimilarity)

<module 'similarities.cosine' from '../src/similarities/cosine.py'>

In [3]:
london_corpus = dataReader.CorpusByNewLine.london()
zwickau_corpus = dataReader.CorpusByNewLine.zwickau()

In [4]:
featuresFactory  = thesisFactoryFeatures.FeaturesFactory(
    london_corpus = london_corpus,
    zwickau_corpus = zwickau_corpus
)

In [5]:
london_zwickau_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(london_corpus, zwickau_corpus)
# london_zwickau_similarities.calculate()

zwickau_london_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(zwickau_corpus, london_corpus)
# zwickau_london_similarities.calculate()

In [38]:
# london_zwickau_similarities.save()
# zwickau_london_similarities.save()
london_zwickau_similarities.load()
zwickau_london_similarities.load()

In [39]:
burchard_corpus_by_london = dataReader.BurchardCorpus(london_corpus, zwickau_corpus)
burchard_corpus_by_zwickau = dataReader.BurchardCorpus(zwickau_corpus, london_corpus)

In [40]:
london_leftovers = dataReader.LeftoversCorpus(london_corpus, zwickau_corpus)
zwickau_leftovers = dataReader.LeftoversCorpus(zwickau_corpus, london_corpus)

In [41]:
print(len(london_leftovers.corpus_for_predictions()))
print(len(zwickau_leftovers.corpus_for_predictions()))

157
146


In [42]:
def filter_short_p(corpus):
    return list(filter(lambda x: len(x.split()) > 20, corpus))

In [43]:
def build_burchard_corpus_with_predictions(burchard_corpus, wrong_predictions_by_london, wrong_predictions_by_zwickau):
    is_burchard = True
    temp_corpus = [ [p, is_burchard, is_burchard] for p in burchard_corpus ]
    
    for prediction in wrong_predictions_by_london:
        temp_corpus[prediction.index][1] = False
    for prediction in wrong_predictions_by_zwickau:
        temp_corpus[prediction.index][2] = False

    return temp_corpus

In [44]:
london_leftofvers_long = filter_short_p(london_leftovers.corpus)
zwickau_leftofvers_long = filter_short_p(zwickau_leftovers.corpus)
burchard_corpus_by_london_corpus_long = burchard_corpus_by_london.corpus
burchard_corpus_by_zwickau_corpus_long = burchard_corpus_by_zwickau.corpus

In [45]:
print(len(burchard_corpus_by_london_corpus_long))
print(len(burchard_corpus_by_zwickau_corpus_long))

217
217


In [18]:
burchard_by_london_VS_zwickau_features_df = thesisModelFeatures.create_features_df(
    london_corpus = None,
    zwickau_corpus = zwickau_leftofvers_long,
    burchard_corpus = burchard_corpus_by_london_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)
burchard_by_zwickau_VS_zwickau_features_df = thesisModelFeatures.create_features_df(
    london_corpus = None,
    zwickau_corpus = zwickau_leftofvers_long,
    burchard_corpus = burchard_corpus_by_zwickau_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)
burchard_by_london_VS_london_features_df = thesisModelFeatures.create_features_df(
    london_corpus = london_leftofvers_long,
    zwickau_corpus = None,
    burchard_corpus = burchard_corpus_by_london_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)
burchard_by_zwickau_VS_london_features_df = thesisModelFeatures.create_features_df(
    london_corpus = london_leftofvers_long,
    zwickau_corpus = None,
    burchard_corpus = burchard_corpus_by_zwickau_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)

n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram


# london vs zwickau 3 ngram

In [55]:
london_vs_zwickau_3features_df = featuresFactory.london_original_VS_zwickau_original(n_gram = 3, features = { 'tfidf' })
london_vs_zwickau_3modesl_experiment = thesisModelFeatures.ModelsExperiment(london_vs_zwickau_3features_df, 'london_vs_zwickau')
london_vs_zwickau_3modesl_experiment.run_cross_validate()
london_vs_zwickau_3modesl_experiment.show_cross_validate_results()

n_gram_feature_name: 3_gram
n_gram_feature_name: 3_gram
running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

Elapsed time to compute the cross validate: 0.038 minutes
running: SVM_RBF
Elapsed time to compute the cross validate: 0.041 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.030 minutes
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

Elapsed time to compute the cross validate: 0.379 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.030 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.210 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.059 minutes
running: MLPClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  war

Elapsed time to compute the cross validate: 4.451 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.007 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.013 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.163 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 1.127 minutes


,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
DecisionTreeClassifier,0.557409,0.553127,0.547532,0.553299,0.547649,0.553299
AdaBoostClassifier,0.551918,0.549371,0.546329,0.549950,0.546607,0.549950
XGBClassifier,0.538528,0.538421,0.536642,0.538938,0.536961,0.538938
MLPClassifier,0.528926,0.528420,0.521251,0.527976,0.521081,0.527976
RandomForestClassifier,0.524509,0.522900,0.515080,0.523289,0.515330,0.523289
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianProcessClassifier,0.491943,0.491709,0.479785,0.492138,0.480000,0.492138
GaussianNB,0.490276,0.492879,0.485533,0.492039,0.485086,0.492039
QuadraticDiscriminantAnalysis,0.456637,0.458813,0.454739,0.459127,0.454974,0.459127
KNeighborsClassifier,0.456118,0.457704,0.451670,0.457391,0.451540,0.457391


## london VS zwickau

In [51]:
london_vs_zwickau_features_df = featuresFactory.london_original_VS_zwickau_original(n_gram = 5, features = { 'tfidf' })

n_gram_feature_name: 5_gram
n_gram_feature_name: 5_gram


In [52]:
london_vs_zwickau_modesl_experiment = thesisModelFeatures.ModelsExperiment(london_vs_zwickau_features_df, 'london_vs_zwickau')

In [53]:
london_vs_zwickau_modesl_experiment.run_cross_validate()

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

Elapsed time to compute the cross validate: 6.259 minutes
running: SVM_RBF


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Elapsed time to compute the cross validate: 6.793 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.250 minutes
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

Elapsed time to compute the cross validate: 2.782 minutes
running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.228 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 2.486 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.219 minutes
running: MLPClassifier
Elapsed time to compute the cross validate: 36.173 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.104 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.136 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.815 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 3.811 minutes


In [54]:
london_vs_zwickau_modesl_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.595312,0.593171,0.589010,0.593775,0.589323,0.593775
DecisionTreeClassifier,0.563434,0.562013,0.559754,0.562550,0.560027,0.562550
AdaBoostClassifier,0.551770,0.552537,0.548970,0.553075,0.549284,0.553075
RandomForestClassifier,0.519794,0.520074,0.515405,0.520164,0.515483,0.520164
MLPClassifier,0.516781,0.517334,0.511935,0.516915,0.511809,0.516915
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
QuadraticDiscriminantAnalysis,0.493073,0.493565,0.486529,0.493502,0.486470,0.493502
GaussianProcessClassifier,0.474614,0.478453,0.467576,0.479415,0.468164,0.479415
GaussianNB,0.474014,0.469937,0.441520,0.468576,0.440777,0.468576
KNeighborsClassifier,0.462399,0.462341,0.457131,0.462202,0.457097,0.462202


In [23]:
london_vs_zwickau_modesl_experiment.run_greed_search_cv_for_n_best_models(3)

running: DecisionTreeClassifier
Elapsed time to compute the greed search cv: 0.488 minutes
running: AdaBoostClassifier
Elapsed time to compute the greed search cv: 24.958 minutes
running: XGBClassifier
Elapsed time to compute the greed search cv: 35.147 minutes


In [24]:
for i in london_vs_zwickau_modesl_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

0.6638709677419354
DecisionTreeClassifier(max_depth=7)
0.6536559139784945
AdaBoostClassifier(learning_rate=0.01, n_estimators=300)
0.6770967741935484
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.4, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=5,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [25]:
london_vs_zwickau_modesl_experiment.greed_search_cv_results[2].best_params_

{'gamma': 0.4, 'max_depth': 3, 'min_child_weight': 5}

## burchard VS zwickau

In [27]:
burchard_by_london_VS_zwickau_models_experiment = thesisModelFeatures.ModelsExperiment(
    burchard_by_london_VS_zwickau_features_df,
    'burchard_by_london_VS_zwickau'
)

In [28]:
burchard_by_london_VS_zwickau_models_experiment.run_cross_validate()

running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.138 minutes
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Elapsed time to compute the cross validate: 0.947 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.117 minutes
running: MLPClassifier
Elapsed time to compute the cross validate: 9.132 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.073 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.097 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.670 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.410 minutes


In [29]:
burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.810930,0.786299,0.792773,0.807207,0.803149,0.807207
AdaBoostClassifier,0.727981,0.718377,0.718092,0.730030,0.728747,0.730030
RandomForestClassifier,0.739112,0.674242,0.671326,0.718994,0.695116,0.718994
DecisionTreeClassifier,0.702478,0.691840,0.692653,0.710435,0.706556,0.710435
GaussianProcessClassifier,0.739324,0.559286,0.489480,0.644520,0.544419,0.644520
MLPClassifier,0.633516,0.577987,0.564658,0.630856,0.597316,0.630856
KNeighborsClassifier,0.612916,0.532879,0.476318,0.611336,0.528870,0.611336
GaussianNB,0.599722,0.520087,0.461203,0.597823,0.514129,0.597823


In [30]:
burchard_by_london_VS_zwickau_models_experiment.run_greed_search_cv_for_n_best_models(3)

running: RandomForestClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.71861862 0.71291291 0.72417417 0.71869369 0.72972973 0.72154655
 0.72695195 0.72972973 0.71861862 0.71291291 0.72417417 0.71869369
 0.72972973 0.72154655 0.72695195 0.72972973 0.59782282 0.59782282
 0.59782282 0.59782282 0.59782282 0.59782282 0.59782282 0.59782282
 0.73235736 0.73798799 0.73528529 0.72695195 0.72695195 0.73250751
 0.7298048  0.73528529 0.73235736 0.73798799 0.73528529 0.72695195
 0.72695195 0.73250751 0.7298048  0.73528529 0.59782282 0.59782282
 0.59782282 0.59782282 0.59782282 0.59782282 0.59782282 0.59782282
 0.72154655 0.7240991  0.74069069 0.72957958 0.73528529 0.73528529
 0.73521021 0.74361862 0.72154655 0.7240991  0.74069069 0.72957958
 0.73528529 0.73528529 0.73521021 0.74361862 0.59496997 0.6033033
 0.60052553 0.59782282 0.59782282 0.59782282 0.59782282 0.59782282
 0.7213964  0.73498498 0.74069069

Elapsed time to compute the greed search cv: 14.043 minutes
running: AdaBoostClassifier
Elapsed time to compute the greed search cv: 56.251 minutes
running: XGBClassifier
Elapsed time to compute the greed search cv: 63.243 minutes


In [31]:
for i in burchard_by_london_VS_zwickau_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

0.7601351351351353
RandomForestClassifier(max_depth=10, n_estimators=500, random_state=0)
0.8071321321321321
AdaBoostClassifier(learning_rate=0.5, n_estimators=500)
0.8067567567567568
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=3,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [32]:
burchard_by_zwickau_VS_zwickau_models_experiment = thesisModelFeatures.ModelsExperiment(burchard_by_zwickau_VS_zwickau_features_df)

TypeError: __init__() missing 1 required positional argument: 'experiment_name'

In [ ]:
burchard_by_zwickau_VS_zwickau_models_experiment.run_cross_validate()

In [ ]:
burchard_by_zwickau_VS_zwickau_models_experiment.show_cross_validate_results()

In [ ]:
burchard_by_zwickau_VS_zwickau_models_experiment.run_greed_search_cv_for_n_best_models(3)

In [ ]:
for i in burchard_by_zwickau_VS_zwickau_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

In [ ]:
burchard_by_zwickau_VS_zwickau_models_experiment.greed_search_cv_results[2].best_params_

## burchard vs london

In [ ]:
burchard_by_london_VS_london_models_experiment = thesisModelFeatures.ModelsExperiment(burchard_by_london_VS_london_features_df)

In [ ]:
burchard_by_london_VS_london_models_experiment.run_cross_validate()

In [ ]:
burchard_by_london_VS_london_models_experiment.show_cross_validate_results()

In [ ]:
burchard_by_london_VS_london_models_experiment.run_greed_search_cv_for_n_best_models(3)

In [ ]:
for i in burchard_by_london_VS_london_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

In [ ]:
burchard_by_zwickau_VS_london_models_experiment = thesisModelFeatures.ModelsExperiment(
    burchard_by_zwickau_VS_london_features_df
)

In [ ]:
burchard_by_zwickau_VS_london_models_experiment.run_cross_validate()

In [ ]:
burchard_by_zwickau_VS_london_models_experiment.show_cross_validate_results()

In [ ]:
burchard_by_zwickau_VS_london_models_experiment.run_greed_search_cv_for_n_best_models(3)

In [ ]:
for i in burchard_by_zwickau_VS_london_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

In [ ]:
# burchard_by_london_VS_zwickau_models_experiment.get_n_best_models(3)

In [ ]:
t= burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results().index[:3]

In [ ]:
t

In [ ]:
best_models_names = burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results().index[:3]
best_models = []

for m, name, greed_search_cv_params in burchard_by_london_VS_zwickau_models_experiment.models:
    if name in best_models_names: best_models.append(m)
best_models

In [ ]:
for m, classifier_name, greed_search_cv_params in best_models:
    print(classifier_name)

In [ ]:
for i in burchard_by_london_VS_zwickau_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

In [ ]:
burchard_by_london_VS_zwickau_models_results = thesisModelFeatures.run_models(
    burchard_by_london_VS_zwickau_features_df
)

In [ ]:
burchard_by_london_VS_zwickau_models_results[0].sort_values(by=['accuracy'], ascending=False)

In [ ]:
'XGBClassifiers' in burchard_by_london_VS_zwickau_models_results[0].sort_values(by=['accuracy'], ascending=False).index[:3]

In [ ]:
burchard_by_zwickau_VS_zwickau_models_results = thesisModelFeatures.run_models(
    burchard_by_zwickau_VS_zwickau_features_df
)

In [ ]:
burchard_by_zwickau_VS_zwickau_models_results[0].sort_values(by=['accuracy'], ascending=False)

## burchard VS london

In [ ]:
burchard_by_london_VS_london_models_results = thesisModelFeatures.run_models(
    burchard_by_london_VS_london_features_df
)

In [ ]:
burchard_by_london_VS_london_models_results[0].sort_values(by=['accuracy'], ascending=False)

In [ ]:
burchard_by_zwickau_VS_london_models_results = thesisModelFeatures.run_models(
    burchard_by_zwickau_VS_london_features_df
)

In [ ]:
burchard_by_zwickau_VS_london_models_results[0].sort_values(by=['accuracy'], ascending=False)

## burchard VS zwickau greed search CV

In [ ]:
# burchard_by_london_VS_zwickau_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_london_VS_zwickau_features_df, [cls])
#     burchard_by_london_VS_zwickau_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

In [ ]:
# burchard_by_london_VS_zwickau_greed_rearch_results

In [ ]:
# burchard_by_zwickau_VS_zwickau_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_zwickau_VS_zwickau_features_df, [cls])
#     burchard_by_zwickau_VS_zwickau_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

In [ ]:
# burchard_by_zwickau_VS_zwickau_greed_rearch_results

## burchard VS london greed search CV

In [ ]:
# burchard_by_london_VS_london_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_london_VS_london_features_df, [cls])
#     burchard_by_london_VS_london_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

In [ ]:
# burchard_by_london_VS_london_greed_rearch_results

In [ ]:
# burchard_by_zwickau_VS_london_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_zwickau_VS_london_features_df, [cls])
#     burchard_by_zwickau_VS_london_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

## testing

In [ ]:
m = thesisModelFeatures.Model(xgb.XGBClassifier())

In [ ]:
m2 = thesisModelFeatures.Model(AdaBoostClassifier(learning_rate=1, n_estimators=2000))